# Multimodal Fusion CRNN @ DHG-14/28

This notebook is for running the 20-fold cross validation experiment using the multimodal fusion CRNN model on the DHG-14/28 dataset.

## Setup

### Secrets
We will track the experiment with [Weights & Biases](https://wandb.ai/). In the Add-ons toolbar in Kaggle, please add your wandb api token as a Secret, which can then be used in the notebook.

In [2]:
from kaggle_secrets import UserSecretsClient
import os

user_secrets = UserSecretsClient()

os.environ["WANDB_API_KEY"] = user_secrets.get_secret("wandb_token")

# sometimes needed in kaggle env
os.environ["LD_LIBRARY_PATH"] = os.environ["LD_LIBRARY_PATH"] + ":/opt/conda/lib"

### Clone the repository

In [13]:
!git clone --single-branch https://github.com/ID56/Multimodal-Fusion-CRNN.git --branch tutorial

Cloning into 'Multimodal-Fusion-CRNN'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 67 (delta 25), reused 30 (delta 7), pack-reused 0
Unpacking objects: 100% (67/67), done.


In [14]:
cd Multimodal-Fusion-CRNN

/kaggle/working/Multimodal-Fusion-CRNN


### Install Requirements

In [15]:
!pip install -qr requirements.txt

## Config

The training and evaluation is controlled by a YAML config file.

In [16]:
config_str = """# sample config

data_root:  /kaggle/input/dhg-14-28
data_list_path: /kaggle/input/dhg-14-28/informations_troncage_sequences.txt

exp:
    wandb: True
    wandb_api_key:
    proj_name: DHG-14-Oct
    exp_dir: /kaggle/working/runs/DHG-14/
    get_stats: False
    cm_path: /kaggle/working/confusionmatrix.png
    tags:
        - featurefusion
    group: feature-0.0.0
    exp_name: 
    device: auto
    log_freq: 20    # log every l_f steps
    val_freq: 1     # validate every v_f epochs; -1 means only at the end
    n_workers: 1
    pin_memory: True
    cache: True
    n_cache_workers: 4
    

hparams:
    seed: 0
    batch_size: 16
    n_epochs: 50
    l_smooth: 0.1
    
    model:
        name:
        type: feature_fusion
        conv_blocks: [8, 16, 32]
        res_in: [50, 50]
        T: 32
        D: 2
        num_classes: 14
        drop_prb: 0.5
        mlp_layers: [128]
        lstm_units: 128
        lstm_layers: 2
        use_bilstm: True
        actn_type: swish
        use_bn: True
        use_ln: False
        
    optimizer:
        opt_type: adamw
        opt_kwargs:
            lr: 0.0003
            weight_decay: 0.1
    
    scheduler:
        n_warmup: 10
        scheduler_type: cosine_annealing

    transforms:
        preprocess:
            gvar:
                g_min: 155
                g_max: 255
                eta: 10
                near_depth_thresh: 200
            
            resize:
                H_new: 50
                W_new: 50
        aug:
            joint_shift_scale_rotate:
                shift_limit: 0.2
                scale_limit: 0.2
                rotate_limit: 20
            
            image_shift_scale_rotate:
                shift_limit: 0.2
                scale_limit: 0.2
                rotate_limit: 20
            
            time_shift:
                p: 0.5    
                frame_limit: 3"""

with open("config.yaml", "w+") as f:
    f.write(config_str)

# Training

Uncomment and run the cell below to initiate training.

In [ ]:
# !python train_DHG.py --conf config.yaml

# Saving CKPT

After training is complete, we can zip up our runs directory so that we can download it after the Kaggle notebook finishes running.

In [ ]:
cd /kaggle/working

In [ ]:
!rm -r Multimodal-Fusion-CRNN

In [ ]:
!zip -r ckpts.zip runs/

In [ ]:
!rm -r runs

# Standalone Evaluation

You can also run evaluation without training, if you have checkpoints available.

Let us first copy checkpoints to the /kaggle/working/runs folder (it is set as the exp_dir in the config file).

In [17]:
!cp -r /kaggle/input/dhgckptoctober/ckpts/runs /kaggle/working/runs

In [20]:
!python eval_DHG.py --conf config.yaml

Set seed 0
100%|███████████████████████████████████████| 2800/2800 [01:56<00:00, 24.12it/s]
Found API key from env variable.
wandb: Currently logged in as: dealer56 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.5
wandb: Syncing run sub_1
wandb: ⭐️ View project at https://wandb.ai/dealer56/DHG-14-Oct
wandb: 🚀 View run at https://wandb.ai/dealer56/DHG-14-Oct/runs/1rewy531
wandb: Run data is saved locally in /kaggle/working/Multimodal-Fusion-CRNN/wandb/run-20221024_072943-1rewy531
wandb: Run `wandb offline` to turn off syncing.

Created model with 4398822 parameters.
Successfully restored state.

CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.

Initiating evaluation.
10